In [7]:
import os
import pandas as pd
import numpy as np

target = '../Satun-Phang-Nga/forcasting2/'

# Constants
gamma = 0.665 * 10**-3  # Psychrometric constant (kPa/°C)

def calculate_delta(Tmean):
    return (4098 * (0.6108 * np.exp((17.27 * Tmean) / (Tmean + 237.3)))) / (Tmean + 237.3)**2

def calculate_saturation_vapor_pressure(Tmean):
    return 0.6108 * np.exp((17.27 * Tmean) / (Tmean + 237.3))

def calculate_actual_vapor_pressure(RH, Tmean):
    e_s = calculate_saturation_vapor_pressure(Tmean)
    return (RH / 100) * e_s

# Process each file
for file in os.listdir(target):
    file_path = os.path.join(target, file)
    df = pd.read_excel(file_path)

    # Extract relevant columns
    Tmean = df['meanTemp']
    RH = df['RH']
    R_s = df['ALLSKY_SFC_SW_DWN']
    u = df['WS']
    P = df['PS']

    # Calculate Delta and e_s
    delta = calculate_delta(Tmean)
    e_s = calculate_saturation_vapor_pressure(Tmean)
    e_a = calculate_actual_vapor_pressure(RH, Tmean)

    # Calculate ETo
    G = 0  # Assumed to be 0 for daily calculations
    ETo = (0.408 * delta * (R_s - G) + gamma * (900 / (Tmean + 273)) * u * (e_s - e_a)) / (delta + gamma * (1 + 0.34 * u))
    df['ETo'] = ETo

    # Calculate WE and WD
    df['WE'] = np.where(P > ETo, P - ETo, 0)  # Water Excess
    df['WD'] = np.where(ETo > P, ETo - P, 0)  # Water Deficit

    # Save the updated dataframe
    output_path = os.path.join(target, file)
    df.to_excel(output_path, index=False)
